In [19]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import re

# Spotify API kimlik bilgilerini ayarlayın
client_id = 'a'
client_secret = 'b'

# Spotify API ile bağlantı kurun
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

# Sanatçı profil linkinden sanatçı ID'sini çıkaran fonksiyon
def extract_artist_id(artist_url):
    pattern = r"spotify:artist:([a-zA-Z0-9]+)|artist/([a-zA-Z0-9]+)"
    match = re.search(pattern, artist_url)
    if match:
        return match.group(1) if match.group(1) else match.group(2)
    else:
        raise ValueError("Invalid Spotify artist URL")

# Belirli bir sanatçıya benzer sanatçıları öneren fonksiyon
def get_similar_artists(artist_url, limit=10):
    artist_id = extract_artist_id(artist_url)
    similar_artists = sp.artist_related_artists(artist_id)
    
    recommendations = []
    for artist in similar_artists['artists'][:limit]:
        recommendations.append(artist['name'])
    
    return recommendations

# Örnek kullanım
artist_url = 'https://open.spotify.com/intl-tr/artist/1bzmtVU7jf2rRZJALkZA3j'
recommended_artists = get_similar_artists(artist_url)
print("\nBenzer Sanatçılar:")
for artist in recommended_artists:
    print(f"- {artist}")



Benzer Sanatçılar:
- yirmi7
- Bağzıları
- Kendimden Hallice
- Perdenin Ardındakiler
- Batuhan Kordel
- Kolpa
- Birileri
- Seksendört
- Yedinci Ev
- Vera


In [12]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import re

# Spotify API kimlik bilgilerini ayarlayın
client_id = 'c'
client_secret = 'v'

# Spotify API ile bağlantı kurun
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

# Sanatçı profil linkinden sanatçı ID'sini çıkaran fonksiyon
def extract_artist_id(artist_url):
    pattern = r"spotify:artist:([a-zA-Z0-9]+)|artist/([a-zA-Z0-9]+)"
    match = re.search(pattern, artist_url)
    if match:
        return match.group(1) if match.group(1) else match.group(2)
    else:
        raise ValueError("Invalid Spotify artist URL")

# Belirli bir sanatçının Spotify popülaritesini hesaplayan fonksiyon
def calculate_spotify_popularity(artist_url):
    artist_id = extract_artist_id(artist_url)
    
    # Sanatçı bilgilerini al
    artist = sp.artist(artist_id)
    followers = artist['followers']['total']
    popularity = artist['popularity']
    
    # Sanatçının en popüler şarkılarını al
    top_tracks = sp.artist_top_tracks(artist_id, country='TR')
    
    # Toplam popülariteyi hesapla
    total_popularity = sum(track['popularity'] for track in top_tracks['tracks'])
    
    return {
        'artist_name': artist['name'],
        'followers': followers,
        'popularity': popularity,
        'total_popularity': total_popularity
    }

# Belirli bir sanatçının en popüler şarkılarını alan fonksiyon
def get_top_tracks(artist_url, country='TR'):
    artist_id = extract_artist_id(artist_url)
    top_tracks = sp.artist_top_tracks(artist_id, country=country)
    
    tracks_info = []
    for track in top_tracks['tracks']:
        tracks_info.append({
            'name': track['name'],
            'popularity': track['popularity'],
            'preview_url': track['preview_url']
        })
    
    return tracks_info

# Sanatçının tüm albümlerini getiren fonksiyon
def get_artist_albums(artist_url):
    artist_id = extract_artist_id(artist_url)
    albums = sp.artist_albums(artist_id, album_type='album')
    
    albums_info = []
    for album in albums['items']:
        albums_info.append({
            'name': album['name'],
            'release_date': album['release_date'],
            'total_tracks': album['total_tracks'],
            'external_url': album['external_urls']['spotify']
        })
    
    return albums_info

# Benzer sanatçıları öneren fonksiyon
def get_similar_artists(artist_url, limit=10):
    artist_id = extract_artist_id(artist_url)
    similar_artists = sp.artist_related_artists(artist_id)
    
    recommendations = []
    for artist in similar_artists['artists'][:limit]:
        recommendations.append(artist['name'])
    
    return recommendations

# Son 15 gün içindeki yeni şarkıları ve popülaritelerini alan fonksiyon
def get_recent_releases(artist_url, days=15):
    artist_id = extract_artist_id(artist_url)
    albums = sp.artist_albums(artist_id, album_type='single')
    recent_tracks = []

    # Güncel tarihi al ve 15 gün öncesine ayarla
    from datetime import datetime, timedelta
    cutoff_date = datetime.now() - timedelta(days=days)

    for album in albums['items']:
        release_date = datetime.strptime(album['release_date'], '%Y-%m-%d')
        if release_date > cutoff_date:
            tracks = sp.album_tracks(album['id'])
            for track in tracks['items']:
                track_info = sp.track(track['id'])
                recent_tracks.append({
                    'name': track_info['name'],
                    'popularity': track_info['popularity'],
                    'release_date': album['release_date']
                })

    return recent_tracks

# Örnek kullanım
artist_url = 'https://open.spotify.com/artist/1dsKaRPU3HFSdlNyMmH5QI'
popularity_data = calculate_spotify_popularity(artist_url)
top_tracks = get_top_tracks(artist_url)
artist_albums = get_artist_albums(artist_url)
recent_releases = get_recent_releases(artist_url)

print(f"Sanatçı Adı: {popularity_data['artist_name']}")
print(f"Takipçi Sayısı: {popularity_data['followers']}")
print(f"Spotify Popülaritesi: {popularity_data['popularity']}")
print(f"Toplam Popülarite Skoru: {popularity_data['total_popularity']}\n")

print("En Popüler Şarkılar:")
for track in top_tracks:
    print(f"- {track['name']} (Popülarite: {track['popularity']})")

print("\nSanatçının Albümleri:")
for album in artist_albums:
    print(f"- {album['name']} (Çıkış Tarihi: {album['release_date']}, Toplam Parça: {album['total_tracks']})")

print("\nSon 15 Gün İçindeki Yeni Şarkılar:")
for track in recent_releases:
    print(f"- {track['name']} (Popülarite: {track['popularity']}, Çıkış Tarihi: {track['release_date']})")


Sanatçı Adı: Fatma Turgut
Takipçi Sayısı: 485287
Spotify Popülaritesi: 50
Toplam Popülarite Skoru: 454

En Popüler Şarkılar:
- Kalbe Zarar (Popülarite: 60)
- İkimizden Biri (Popülarite: 55)
- Hata (Popülarite: 52)
- İlkbaharda Kıyamet (Popülarite: 48)
- Bir Varmış Bir Yokmuş (Popülarite: 46)
- Dönmem Ona (Popülarite: 42)
- Bensiz (Popülarite: 39)
- Sen Diyorum (Popülarite: 39)
- Ben Vardım (Popülarite: 38)
- Beni Tutmayın (Popülarite: 35)

Sanatçının Albümleri:
- Elimde Dünya (Çıkış Tarihi: 2019-05-17, Toplam Parça: 10)

Son 15 Gün İçindeki Yeni Şarkılar:


In [15]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import re

# Spotify API kimlik bilgilerini ayarlayın
client_id = 'd'
client_secret = 'e'

# Spotify API ile bağlantı kurun
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

# Sanatçı profil linkinden sanatçı ID'sini çıkaran fonksiyon
def extract_artist_id(artist_url):
    pattern = r"spotify:artist:([a-zA-Z0-9]+)|artist/([a-zA-Z0-9]+)"
    match = re.search(pattern, artist_url)
    if match:
        return match.group(1) if match.group(1) else match.group(2)
    else:
        raise ValueError("Invalid Spotify artist URL")

# Belirli bir sanatçının Spotify popülaritesini hesaplayan fonksiyon
def calculate_spotify_popularity(artist_url):
    artist_id = extract_artist_id(artist_url)
    
    # Sanatçı bilgilerini al
    artist = sp.artist(artist_id)
    followers = artist['followers']['total']
    popularity = artist['popularity']
    
    # Sanatçının en popüler şarkılarını al
    top_tracks = sp.artist_top_tracks(artist_id, country='TR')
    
    # Toplam popülariteyi hesapla
    total_popularity = sum(track['popularity'] for track in top_tracks['tracks'])
    
    return {
        'artist_name': artist['name'],
        'followers': followers,
        'popularity': popularity,
        'total_popularity': total_popularity
    }

# Belirli bir sanatçının en popüler şarkılarını alan fonksiyon
def get_top_tracks(artist_url, country='TR'):
    artist_id = extract_artist_id(artist_url)
    top_tracks = sp.artist_top_tracks(artist_id, country=country)
    
    tracks_info = []
    for track in top_tracks['tracks']:
        tracks_info.append({
            'name': track['name'],
            'popularity': track['popularity'],
            'preview_url': track['preview_url']
        })
    
    return tracks_info

# Sanatçının tüm albümlerini getiren fonksiyon
def get_artist_albums(artist_url):
    artist_id = extract_artist_id(artist_url)
    albums = sp.artist_albums(artist_id, album_type='album')
    
    albums_info = []
    for album in albums['items']:
        albums_info.append({
            'name': album['name'],
            'release_date': album['release_date'],
            'total_tracks': album['total_tracks'],
            'external_url': album['external_urls']['spotify']
        })
    
    return albums_info

# Benzer sanatçıları öneren fonksiyon
def get_similar_artists(artist_url, limit=10):
    artist_id = extract_artist_id(artist_url)
    similar_artists = sp.artist_related_artists(artist_id)
    
    recommendations = []
    for artist in similar_artists['artists'][:limit]:
        recommendations.append(artist['name'])
    
    return recommendations

# Son 15 gün içindeki yeni şarkıları ve popülaritelerini alan fonksiyon
def get_recent_releases(artist_url, days=15):
    artist_id = extract_artist_id(artist_url)
    albums = sp.artist_albums(artist_id, album_type='single')
    recent_tracks = []

    # Güncel tarihi al ve 15 gün öncesine ayarla
    from datetime import datetime, timedelta
    cutoff_date = datetime.now() - timedelta(days=days)

    for album in albums['items']:
        release_date = datetime.strptime(album['release_date'], '%Y-%m-%d')
        if release_date > cutoff_date:
            tracks = sp.album_tracks(album['id'])
            for track in tracks['items']:
                track_info = sp.track(track['id'])
                recent_tracks.append({
                    'name': track_info['name'],
                    'popularity': track_info['popularity'],
                    'release_date': album['release_date']
                })

    return recent_tracks

# Örnek kullanım
artist_url = 'https://open.spotify.com/artist/1dsKaRPU3HFSdlNyMmH5QI'
popularity_data = calculate_spotify_popularity(artist_url)
top_tracks = get_top_tracks(artist_url)
artist_albums = get_artist_albums(artist_url)
recommended_artists = get_similar_artists(artist_url)
recent_releases = get_recent_releases(artist_url)

print(f"Sanatçı Adı: {popularity_data['artist_name']}")
print(f"Takipçi Sayısı: {popularity_data['followers']}")
print(f"Spotify Popülaritesi: {popularity_data['popularity']}")
print(f"Toplam Popülarite Skoru: {popularity_data['total_popularity']}\n")

print("En Popüler Şarkılar:")
for track in top_tracks:
    print(f"- {track['name']} (Popülarite: {track['popularity']})")

print("\nSanatçının Albümleri:")
for album in artist_albums:
    print(f"- {album['name']} (Çıkış Tarihi: {album['release_date']}, Toplam Parça: {album['total_tracks']})")

print("\nBenzer Sanatçılar:")
for artist in recommended_artists:
    print(f"- {artist}")

print("\nSon 15 Gün İçindeki Yeni Şarkılar:")
for track in recent_releases:
    print(f"- {track['name']} (Popülarite: {track['popularity']}, Çıkış Tarihi: {track['release_date']})")


Sanatçı Adı: Fatma Turgut
Takipçi Sayısı: 485287
Spotify Popülaritesi: 50
Toplam Popülarite Skoru: 454

En Popüler Şarkılar:
- Kalbe Zarar (Popülarite: 60)
- İkimizden Biri (Popülarite: 55)
- Hata (Popülarite: 52)
- İlkbaharda Kıyamet (Popülarite: 48)
- Bir Varmış Bir Yokmuş (Popülarite: 46)
- Dönmem Ona (Popülarite: 42)
- Bensiz (Popülarite: 39)
- Sen Diyorum (Popülarite: 39)
- Ben Vardım (Popülarite: 38)
- Beni Tutmayın (Popülarite: 35)

Sanatçının Albümleri:
- Elimde Dünya (Çıkış Tarihi: 2019-05-17, Toplam Parça: 10)

Benzer Sanatçılar:
- yirmi7
- Eda Baba
- Kaan Tangöze
- Farah Zeynep Abdullah
- Ferman Akgül
- Bağzıları
- Model
- Pera
- Ufuk Beydemir
- Tuğkan

Son 15 Gün İçindeki Yeni Şarkılar:


In [26]:
import googleapiclient.discovery
import re
from datetime import datetime

# YouTube API kimlik bilgilerini ayarlayın
api_key = 'f'

# YouTube API istemcisini oluşturun
youtube = googleapiclient.discovery.build('youtube', 'v3', developerKey=api_key)

# Belirli bir arama terimi ile YouTube'da video araması yapan fonksiyon
def search_videos(query, max_results=10):
    request = youtube.search().list(
        part='snippet',
        q=query,
        type='video',
        maxResults=max_results
    )
    response = request.execute()
    
    videos = []
    for item in response['items']:
        videos.append({
            'title': item['snippet']['title'],
            'video_id': item['id']['videoId'],
            'description': item['snippet']['description'],
            'published_at': item['snippet']['publishedAt'],
            'channel_id': item['snippet']['channelId']
        })
    
    return videos

# Video ID'sine göre video istatistiklerini alan fonksiyon
def get_video_statistics(video_id):
    request = youtube.videos().list(
        part='statistics',
        id=video_id
    )
    response = request.execute()
    if 'items' not in response or not response['items']:
        raise ValueError("Video not found")
    
    video_stats = response['items'][0]['statistics']
    return {
        'view_count': int(video_stats.get('viewCount', 0)),
        'like_count': int(video_stats.get('likeCount', 0)),
        'comment_count': int(video_stats.get('commentCount', 0))
    }

# Belirli bir arama terimi ile ilgili videoların analizini yapan fonksiyon
def analyze_videos(query, max_results=10):
    videos = search_videos(query, max_results)
    for video in videos:
        stats = get_video_statistics(video['video_id'])
        video.update(stats)
    return videos

# Belirli bir kanal ID'sine göre en popüler videoları alan fonksiyon
def get_channel_videos(channel_id, max_results=10):
    request = youtube.search().list(
        part='snippet',
        channelId=channel_id,
        order='viewCount',
        type='video',
        maxResults=max_results
    )
    response = request.execute()
    
    videos = []
    for item in response['items']:
        videos.append({
            'title': item['snippet']['title'],
            'video_id': item['id']['videoId'],
            'description': item['snippet']['description'],
            'published_at': item['snippet']['publishedAt']
        })
    
    return videos

# Belirli bir kanal ID'sine göre kanal istatistiklerini alan fonksiyon
def get_channel_statistics(channel_id):
    request = youtube.channels().list(
        part='statistics',
        id=channel_id
    )
    response = request.execute()
    if 'items' not in response or not response['items']:
        raise ValueError("Channel not found")
    
    channel_stats = response['items'][0]['statistics']
    return {
        'subscriber_count': int(channel_stats.get('subscriberCount', 0)),
        'video_count': int(channel_stats.get('videoCount', 0)),
        'view_count': int(channel_stats.get('viewCount', 0))
    }

# Belirli bir sanatçının kanal ve video popülaritesini analiz eden fonksiyon
def analyze_artist_popularity(artist_url, max_results=10):
    artist_id = extract_artist_id(artist_url)
    artist_name = sp.artist(artist_id)['name']
    
    # Kanal araması
    channel_search_results = youtube.search().list(
        part='snippet',
        q=artist_name,
        type='channel',
        maxResults=1
    ).execute()
    
    if 'items' not in channel_search_results or not channel_search_results['items']:
        raise ValueError("Channel not found for the artist")
    
    channel_id = channel_search_results['items'][0]['id']['channelId']
    channel_stats = get_channel_statistics(channel_id)
    
    # Kanalın en popüler videolarını al
    channel_videos = get_channel_videos(channel_id, max_results)
    for video in channel_videos:
        stats = get_video_statistics(video['video_id'])
        video.update(stats)
    
    # Sanatçının toplam popülaritesini hesapla
    total_popularity = sum(video['view_count'] for video in channel_videos)
    
    return {
        'artist_name': artist_name,
        'channel_id': channel_id,
        'subscriber_count': channel_stats['subscriber_count'],
        'video_count': channel_stats['video_count'],
        'total_channel_views': channel_stats['view_count'],
        'total_video_popularity': total_popularity,
        'popular_videos': channel_videos
    }

# Sanatçı URL'sinden sanatçı ID'sini çıkaran fonksiyon
def extract_artist_id(artist_url):
    pattern = r"spotify:artist:([a-zA-Z0-9]+)|artist/([a-zA-Z0-9]+)"
    match = re.search(pattern, artist_url)
    if match:
        return match.group(1) if match.group(1) else match.group(2)
    else:
        raise ValueError("Invalid Spotify artist URL")

# Örnek kullanım
artist_url = 'https://open.spotify.com/artist/1dsKaRPU3HFSdlNyMmH5QI'
artist_popularity = analyze_artist_popularity(artist_url)

print(f"Sanatçı Adı: {artist_popularity['artist_name']}")
print(f"Abone Sayısı: {artist_popularity['subscriber_count']}")
print(f"Video Sayısı: {artist_popularity['video_count']}")
print(f"Toplam Kanal İzlenme Sayısı: {artist_popularity['total_channel_views']}")
print(f"Toplam Video Popülaritesi: {artist_popularity['total_video_popularity']}")

print(f"{artist_popularity['artist_name']} ile ilgili en popüler videolar:")
for video in artist_popularity['popular_videos']:
    print(f"- {video['title']} (İzlenme Sayısı: {video['view_count']}, Beğeni Sayısı: {video['like_count']}, Yorum Sayısı: {video['comment_count']}) - https://www.youtube.com/watch?v={video['video_id']}")


Sanatçı Adı: Fatma Turgut
Abone Sayısı: 52700
Video Sayısı: 38
Toplam Kanal İzlenme Sayısı: 14281835
Toplam Video Popülaritesi: 5014839
Fatma Turgut ile ilgili en popüler videolar:
- Fatma Turgut - İlkbaharda Kıyamet (IF Konser Videosu) #Canlı (İzlenme Sayısı: 1256570, Beğeni Sayısı: 12611, Yorum Sayısı: 371) - https://www.youtube.com/watch?v=rZmwoM7Pb2U
- Fatma Turgut - Sen Diyorum (Akustik Performans) #Canlı (İzlenme Sayısı: 1096571, Beğeni Sayısı: 15893, Yorum Sayısı: 468) - https://www.youtube.com/watch?v=oFQrRp6FVjY
- Fatma Turgut - Yalnızlık Senfonisi (Akustik Performans) #Canlı (İzlenme Sayısı: 878755, Beğeni Sayısı: 15146, Yorum Sayısı: 1121) - https://www.youtube.com/watch?v=4FwYAluLC0o
- Fatma Turgut - Unuttum Gitti (Akustik Performans) #Canlı (İzlenme Sayısı: 470446, Beğeni Sayısı: 7654, Yorum Sayısı: 620) - https://www.youtube.com/watch?v=nInRtPJrz1c
- Fatma Turgut - Yangın Yeri (Akustik Performans) #Canlı (İzlenme Sayısı: 277362, Beğeni Sayısı: 4782, Yorum Sayısı: 397) - h